# Project Title

## Overview

Our company is making the venture into the airline industry. We have been tasked with determining which aircraft carry the lowest risk. In this notebook, we have provided three recommendation for our company's airplane purchases.

## Business Understanding

stakeholder and business questions

## Data Understanding

Data source and properties and why relevant to problem

The data has been provided in this [Aviation Accident Database 1948-2022](https://www.kaggle.com/datasets/khsamaha/aviation-accident-database-synopses) which is publicly available through [kaggle](https://www.kaggle.com/) datasets.

In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv('../Data/Aviation_Data.csv', parse_dates=['Event.Date', 'Publication.Date'])
df.head()

explore properties of data and why it's well suited to solivng the prob

## Data Preparation

rationalize every step

## Analysis and Results

the groupby's and the visuals produced in tableau embedded following those cells

### Business Recommendation 1

### Business Recommendation 2

### Business Recommendation 3

## Conclusion

### Next Steps